In [ ]:
import cirrus

In [ ]:
basic_params = {
    'n_workers': 5,
    'n_ps': 1,
    'dataset': 'criteo-kaggle',
    'epsilon': 0.0001,
    'key_name': 'mykey',
    'key_path': '~/key.pem',
    'minibatch_size': 20,
    'model_bits': 19,
    'train_set': (0,824),
    'test_set': (835,840)
}


gs = cirrus.GridSearch(task=cirrus.LogisticRegression,
                       param_base=basic_params,
                       hyper_vars=["learning_rate", "minibatch_size"],
                       hyper_params=[[0.1, 0.01, 0.001], [20, 200, 2000]]
                      )
gs.run(UI=True)


In [ ]:
app.display_dash()